In [1]:
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI
from langchain.graphs import Neo4jGraph
from langchain.callbacks import get_openai_callback
from dotenv import load_dotenv
import os
import openai
import pandas as pd
from neo4j.exceptions import CypherSyntaxError


## Choose the LLM

In [2]:
LLM_MODEL = 'gpt-4-32k'


## Custom function for neo4j RAG chain

In [5]:
def get_neo4j_cypher_rag_chain():
    load_dotenv(os.path.join(os.path.expanduser('~'), '.spoke_neo4j_config.env'))
    username = os.environ.get('NEO4J_USER')
    password = os.environ.get('NEO4J_PSW')
    url = os.environ.get('NEO4J_URI')
    database = os.environ.get('NEO4J_DB')

    graph = Neo4jGraph(
        url=url, 
        username=username, 
        password=password,
        database = database
    )

    load_dotenv(os.path.join(os.path.expanduser('~'), '.gpt_config.env'))
    API_KEY = os.environ.get('API_KEY')
    API_VERSION = os.environ.get('API_VERSION')
    RESOURCE_ENDPOINT = os.environ.get('RESOURCE_ENDPOINT')
    openai.api_type = "azure"
    openai.api_key = API_KEY
    openai.api_base = RESOURCE_ENDPOINT
    openai.api_version = API_VERSION
    chat_deployment_id = LLM_MODEL
    chat_model_id = chat_deployment_id
    temperature = 0
    chat_model = ChatOpenAI(openai_api_key=API_KEY, 
                            engine=chat_deployment_id, 
                            temperature=temperature)
    chain = GraphCypherQAChain.from_llm(
        chat_model, 
        graph=graph, 
        verbose=True, 
        validate_cypher=True,
        return_intermediate_steps=True
    )
    return chain

## Initiate neo4j RAG chain

In [6]:
%%time
neo4j_rag_chain = get_neo4j_cypher_rag_chain()


WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


CPU times: user 17.5 ms, sys: 3.65 ms, total: 21.2 ms
Wall time: 28.1 s


## Enter question -1 

In [9]:
question = "Is Parkinson's disease associated with PINK1 gene?"


## Run cypher RAG

In [11]:

out = neo4j_rag_chain.run(query=question, return_final_only=True, verbose=True)
print(out)




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Disease {name: "Parkinson's disease"}), (g:Gene {name: "PINK1"}) 
RETURN EXISTS((d)-[:ASSOCIATES_DaG]->(g)) AS is_associated
Full Context:
[{'is_associated': True}]

> Finished chain.
Yes, Parkinson's disease is associated with the PINK1 gene.


## Question 1 perturbed (all smallcase)

In [18]:
question = "Is parkinson's disease associated with pink1 gene?"


## Run cypher RAG

In [19]:

out = neo4j_rag_chain.run(query=question, return_final_only=True, verbose=True)
print(out)




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Disease {name: "Parkinson's Disease"}), (g:Gene {name: "PINK1"}) 
RETURN EXISTS((d)-[:ASSOCIATES_DaG]->(g)) AS is_associated
Full Context:
[]

> Finished chain.
I'm sorry, but I don't have the information to answer that question.
